<a href="https://colab.research.google.com/github/Deepti-Shringare/Downscaling_of_no2map_XGBoost/blob/main/COLAB_CODES/Copy_of_merging_dataset_part2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import geopandas as gpd
import ast
import os
from shapely.geometry import Point
from google.colab import drive

# ============= MOUNT GOOGLE DRIVE =============
drive.mount('/content/drive')

# --------------------
# STEP 1: Load CPCB daily groundtruth
# --------------------
cpcb = pd.read_csv("/content/drive/MyDrive/DELHI/cpcb_daily.csv")
cpcb["date"] = pd.to_datetime(cpcb["date"])
cpcb["month"] = cpcb["date"].dt.to_period("M")  # YYYY-MM
cpcb_monthly = cpcb.groupby(["Station Name","month"])["NO2"].mean().reset_index()

# --------------------
# STEP 2: Load station locations
# --------------------
stations = pd.read_csv("/content/drive/MyDrive/DELHI/delhi_station_population_2024.csv")
gdf_stations = gpd.GeoDataFrame(
    stations,
    geometry=gpd.points_from_xy(stations["Longitude"], stations["Latitude"]),
    crs="EPSG:4326"
)

# --------------------
# STEP 3: Function to parse one satellite file
# --------------------
def parse_satellite_file(file_path, month_str):
    sat = pd.read_csv(file_path)

    # Parse lat/lon from geo column
    def parse_coords(geo_str):
        try:
            geo_dict = ast.literal_eval(geo_str)
            return geo_dict["coordinates"][0], geo_dict["coordinates"][1]
        except:
            return None, None

    sat[["lon", "lat"]] = sat[".geo"].apply(lambda g: pd.Series(parse_coords(g)))

    gdf_sat = gpd.GeoDataFrame(
        sat,
        geometry=gpd.points_from_xy(sat["lon"], sat["lat"]),
        crs="EPSG:4326"
    )

    # nearest join
    joined = gpd.sjoin_nearest(gdf_stations, gdf_sat, how="left")

    # assign month
    joined["month"] = month_str

    return joined[["STATION NAME", "month", "tropospheric_NO2_column_number_density"]]

# --------------------
# STEP 4: Loop over all satellite monthly files
# --------------------
sat_dir = "/content/drive/MyDrive/DELHI/" # Corrected to directory path
all_sat = []

for file in os.listdir(sat_dir):
    if file.endswith(".csv") and "TROPOMI_NO2_Delhi" in file: # Added check for filename pattern
        # Extract month from filename
        # Example: "TROPOMI_NO2_Delhi_April2024.csv"
        month_name = file.split("_")[-1].replace(".csv","")  # e.g. "April2024"
        month_str = pd.to_datetime(month_name, format="%B%Y").to_period("M")  # YYYY-MM

        print(f"Processing {file} as {month_str}")
        sat_data = parse_satellite_file(os.path.join(sat_dir, file), month_str)
        all_sat.append(sat_data)

satellite_all = pd.concat(all_sat, ignore_index=True)

# --------------------
# STEP 5: Merge CPCB monthly averages with satellite
# --------------------
final = pd.merge(
    cpcb_monthly,
    satellite_all,
    left_on=["Station Name","month"],
    right_on=["STATION NAME","month"],
    how="inner"
)

final.to_csv("/content/drive/MyDrive/DELHI/merged_NO2_station_satellite.csv", index=False)
print("✅ Final merged dataset saved!")

In [ ]:
import pandas as pd
import geopandas as gpd
import ast
import os
from shapely.geometry import Point

# --------------------
# STEP 1: Load CPCB daily groundtruth
# --------------------
cpcb = pd.read_csv("/content/drive/MyDrive/DELHI/cpcb_daily.csv")
cpcb["date"] = pd.to_datetime(cpcb["date"])
cpcb["month"] = cpcb["date"].dt.to_period("M")  # YYYY-MM
cpcb_monthly = cpcb.groupby(["Station Name","month"])["NO2"].mean().reset_index()

# --------------------
# STEP 2: Load station locations
# --------------------
stations = pd.read_csv("/content/drive/MyDrive/DELHI/delhi_station_population_2024.csv")
gdf_stations = gpd.GeoDataFrame(
    stations,
    geometry=gpd.points_from_xy(stations["Longitude"], stations["Latitude"]),
    crs="EPSG:4326"
)

# --------------------
# STEP 3: Function to parse one satellite file
# --------------------
def parse_satellite_file(file_path, month_str):
    sat = pd.read_csv(file_path)

    # Parse lat/lon from geo column
    def parse_coords(geo_str):
        try:
            geo_dict = ast.literal_eval(geo_str)
            return geo_dict["coordinates"][0], geo_dict["coordinates"][1]
        except:
            return None, None

    sat[["lon", "lat"]] = sat[".geo"].apply(lambda g: pd.Series(parse_coords(g)))

    gdf_sat = gpd.GeoDataFrame(
        sat,
        geometry=gpd.points_from_xy(sat["lon"], sat["lat"]),
        crs="EPSG:4326"
    )

    # nearest join
    joined = gpd.sjoin_nearest(gdf_stations, gdf_sat, how="left")

    # assign month
    joined["month"] = month_str

    return joined[["STATION NAME", "month", "tropospheric_NO2_column_number_density"]]

# --------------------
# STEP 4: Loop over all satellite monthly files
# --------------------
sat_dir = "/content/drive/MyDrive/DELHI/Satellite"
all_sat = []

for file in os.listdir(sat_dir):
    if file.endswith(".csv"):
        # Extract month from filename
        # Example: "TROPOMI_NO2_Delhi_April2024.csv"
        month_name = file.split("_")[-1].replace(".csv","")  # e.g. "April2024"
        month_str = pd.to_datetime(month_name, format="%B%Y").to_period("M")  # YYYY-MM

        print(f"Processing {file} as {month_str}")
        sat_data = parse_satellite_file(os.path.join(sat_dir, file), month_str)
        all_sat.append(sat_data)

satellite_all = pd.concat(all_sat, ignore_index=True)

# --------------------
# STEP 5: Merge CPCB monthly averages with satellite
# --------------------
final = pd.merge(
    cpcb_monthly,
    satellite_all,
    left_on=["Station Name","month"],
    right_on=["STATION NAME","month"],
    how="inner"
)

final.to_csv("/content/drive/MyDrive/DELHI/merged_NO2_station_satellite.csv", index=False)
print("✅ Final merged dataset saved!")


In [ ]:
import pandas as pd

# --------------------
# STEP 1: Load CPCB groundtruth
# --------------------
cpcb = pd.read_csv("/content/drive/MyDrive/DELHI/cpcb_daily.csv")
cpcb["date"] = pd.to_datetime(cpcb["date"])

# --------------------
# STEP 2: Load Satellite daily dataset
# --------------------
sat = pd.read_csv("/content/drive/MyDrive/DELHI/Delhi_NO2_Daily_Satellite_2024.csv")
sat["date"] = pd.to_datetime(sat["date"])

# --------------------
# STEP 3: Merge on date
# --------------------
# Rename for clarity
sat = sat.rename(columns={"NO2": "Satellite_NO2"})

# Merge: each station’s daily CPCB NO2 with same day’s satellite mean NO2
merged = pd.merge(
    cpcb,
    sat[["date", "Satellite_NO2"]],
    on="date",
    how="inner"
)

# --------------------
# STEP 4: Save
# --------------------
merged.to_csv("/content/drive/MyDrive/DELHI/merged_NO2_daily.csv", index=False)

print("✅ Final merged daily dataset saved!")
print(merged.head())


In [ ]:
import pandas as pd

# --------------------
# STEP 1: Load merged daily NO2 dataset
# --------------------
merged = pd.read_csv("/content/drive/MyDrive/DELHI/merged_NO2_daily.csv")

# --------------------
# STEP 2: Load population and nightlights datasets
# --------------------
pop = pd.read_csv("/content/drive/MyDrive/DELHI/delhi_station_population_2024.csv")
lights = pd.read_csv("/content/drive/MyDrive/DELHI/delhi_stations_ntl_2024_average.csv")

# Ensure consistent station name column
for df in [pop, lights]:
    if "Station Name" not in df.columns:
        # Rename the 'name' column to 'Station Name' in the lights dataframe
        if "name" in df.columns:
             df.rename(columns={"name": "Station Name"}, inplace=True)


# --------------------
# STEP 3: Merge all datasets
# --------------------
final = merged.merge(pop[["Station Name", "population_2024"]], on="Station Name", how="left")
# Use 'avg_rad' column for nightlights data
final = final.merge(lights[["Station Name", "avg_rad"]], on="Station Name", how="left")
final.rename(columns={"avg_rad": "nightlights_2024"}, inplace=True)


# --------------------
# STEP 4: Save
# --------------------
final.to_csv("/content/drive/MyDrive/DELHI/merged_NO2_population_nightlights.csv", index=False)

print("✅ Final dataset with Population + Nighttime Lights saved!")
print(final.head())

In [ ]:
import pandas as pd

# --------------------
# STEP 1: Load merged daily NO2 dataset
# --------------------
merged = pd.read_csv("/content/drive/MyDrive/DELHI/merged_NO2_daily.csv")

# --------------------
# STEP 2: Load population and nightlights datasets
# --------------------
pop = pd.read_csv("/content/drive/MyDrive/DELHI/delhi_station_population_2024.csv")
lights = pd.read_csv("/content/drive/MyDrive/DELHI/delhi_stations_ntl_2024_average _2.csv")

# Ensure consistent station name column
for df in [pop, lights]:
    if "Station Name" not in df.columns:
        df.rename(columns={col: "Station Name" for col in df.columns if "Station" in col}, inplace=True)

# --------------------
# STEP 3: Merge all datasets
# --------------------
final = merged.merge(pop[["Station Name", "population_2024"]], on="Station Name", how="left")
final = final.merge(lights[["Station Name", "avg_rad"]], on="Station Name", how="left")

# --------------------
# STEP 4: Save
# --------------------
final.to_csv("/content/drive/MyDrive/DELHI/merged_NO2_population_nightlights.csv", index=False)

print("✅ Final dataset with Population + Nighttime Lights saved!")
print(final.head())


In [ ]:
import re
import pandas as pd
from google.colab import drive

# --------------------
# STEP 1: Mount Drive
# --------------------
drive.mount('/content/drive')

# --------------------
# STEP 2: File paths
# --------------------
groundtruth_file = "/content/drive/MyDrive/DELHI/merged_NO2_daily.csv"  # your NO2+satellite merged file
population_file  = "/content/drive/MyDrive/DELHI/delhi_station_population_2024.csv"
night_file       = "/content/drive/MyDrive/DELHI/delhi_stations_ntl_2024_average _2.csv"

output_file      = "/content/drive/MyDrive/DELHI/final_merged_dataset.csv"

# --------------------
# STEP 3: Load datasets
# --------------------
df_no2   = pd.read_csv(groundtruth_file)
df_pop   = pd.read_csv(population_file)
df_night = pd.read_csv(night_file)

# --------------------
# STEP 4: Clean station names
# --------------------
def clean_station_names(df, col="Station Name"):
    """
    Cleans station names by:
    - Removing anything in parentheses e.g. "Alipur (1)" -> "Alipur"
    - Stripping extra spaces
    - Converting to uppercase (so merge is case-insensitive)
    """
    df[col] = df[col].astype(str) \
                     .apply(lambda x: re.sub(r"\s*\(.*\)", "", x)) \
                     .str.strip() \
                     .str.upper()
    return df

df_no2   = clean_station_names(df_no2, "Station Name")
df_pop   = clean_station_names(df_pop, "STATION NAME") # Pass the correct column name for df_pop
df_night = clean_station_names(df_night, "Station Name")

# --------------------
# STEP 5: Merge datasets
# --------------------
# First merge NO2+satellite with population
df_merged = df_no2.merge(df_pop, left_on="Station Name", right_on="STATION NAME", how="left") # Use left_on and right_on for merging
df_merged = df_merged.drop(columns="STATION NAME") # Drop the extra column from the population dataframe

# Then merge with nighttime dataset
df_merged = df_merged.merge(df_night, on="Station Name", how="left")

# --------------------
# STEP 6: Save final dataset
# --------------------
df_merged.to_csv(output_file, index=False)
print(f"✅ Final merged dataset saved at: {output_file}")
print(df_merged.head())

In [ ]:
import re
import pandas as pd
from google.colab import drive

# --------------------
# STEP 1: Mount Drive
# --------------------
drive.mount('/content/drive')

# --------------------
# STEP 2: File paths
# --------------------
groundtruth_file = "/content/drive/MyDrive/DELHI/merged_NO2_daily.csv"  # NO2+satellite merged file
population_file  = "/content/drive/MyDrive/DELHI/delhi_station_population_2024_2.csv"
night_file       = "/content/drive/MyDrive/DELHI/delhi_stations_ntl_2024_average _2.csv"

output_file      = "/content/drive/MyDrive/DELHI/final_merged_dataset_cleaned.csv"

# --------------------
# STEP 3: Load datasets
# --------------------
df_no2   = pd.read_csv(groundtruth_file)
df_pop   = pd.read_csv(population_file)
df_night = pd.read_csv(night_file)

# --------------------
# STEP 4: Clean station names (make consistent)
# --------------------
def clean_station_names(df, col="Station Name"):
    """
    Cleans station names:
    - Removes text in parentheses: 'Alipur (1)' -> 'Alipur'
    - Strips spaces
    - Converts to Title Case (First letter capital, rest lower) for consistency
    """
    df[col] = df[col].astype(str) \
                     .apply(lambda x: re.sub(r"\s*\(.*\)", "", x)) \
                     .str.strip() \
                     .str.title()
    return df

df_no2   = clean_station_names(df_no2, "Station Name")
df_pop   = clean_station_names(df_pop, "Station Name")
df_night = clean_station_names(df_night, "Station Name")

# --------------------
# STEP 5: Drop unnecessary columns
# --------------------
drop_cols = ["Sr No", "system:index", ".geo", "Longitude", "Latitude"]  # adjust if present
for col in drop_cols:
    if col in df_no2.columns:
        df_no2 = df_no2.drop(columns=[col])
    if col in df_pop.columns:
        df_pop = df_pop.drop(columns=[col])
    if col in df_night.columns:
        df_night = df_night.drop(columns=[col])

# --------------------
# STEP 6: Merge datasets
# --------------------
df_merged = df_no2.merge(df_pop, on="Station Name", how="left") # Corrected merge key
df_merged = df_merged.merge(df_night, on="Station Name", how="left")

# --------------------
# STEP 7: Save cleaned dataset
# --------------------
df_merged.to_csv(output_file, index=False)
print(f"✅ Cleaned dataset saved at: {output_file}")
print("Preview:")
print(df_merged.head(20))

In [ ]:
import re
import pandas as pd
from google.colab import drive

# --------------------
# STEP 1: Mount Drive
# --------------------
drive.mount('/content/drive')

# --------------------
# STEP 2: File paths
# --------------------
groundtruth_file = "/content/drive/MyDrive/DELHI/merged_NO2_daily.csv"  # NO2+satellite merged file
population_file  = "/content/drive/MyDrive/DELHI/delhi_station_population_2024_2.csv"
night_file       = "/content/drive/MyDrive/DELHI/delhi_stations_ntl_2024_average _2.csv"

output_file      = "/content/drive/MyDrive/DELHI/final_merged_dataset_cleaned_2.csv"

# --------------------
# STEP 3: Load datasets
# --------------------
df_no2   = pd.read_csv(groundtruth_file)
df_pop   = pd.read_csv(population_file)
df_night = pd.read_csv(night_file)

# --------------------
# STEP 4: Clean station names
# --------------------
def clean_station_names(df, col="Station Name"):
    """
    Cleans station names:
    - Removes parentheses like 'Alipur (1)' -> 'Alipur'
    - Strips spaces
    - Converts to Title Case (Alipur, Wazirpur, etc.)
    """
    df[col] = df[col].astype(str) \
                     .apply(lambda x: re.sub(r"\s*\(.*\)", "", x)) \
                     .str.strip() \
                     .str.title()
    return df

df_no2   = clean_station_names(df_no2, "Station Name")
df_pop   = clean_station_names(df_pop, "Station Name")
df_night = clean_station_names(df_night, "Station Name")

# --------------------
# STEP 5: Drop unnecessary columns
# --------------------
drop_cols = ["SR NO.", "Station category", "system:index", ".geo"]
for df in [df_no2, df_pop, df_night]:
    for col in drop_cols:
        if col in df.columns:
            df.drop(columns=[col], inplace=True)

# --------------------
# STEP 6: Check station names before merge
# --------------------
print("✅ Groundtruth stations:", sorted(df_no2["Station Name"].unique()))
print("✅ Population stations:", sorted(df_pop["Station Name"].unique()))
print("✅ Nighttime stations:", sorted(df_night["Station Name"].unique()))

# --------------------
# STEP 7: Merge datasets
# --------------------
df_merged = df_no2.merge(df_pop, on="Station Name", how="left")
df_merged = df_merged.merge(df_night, on="Station Name", how="left")

# --------------------
# STEP 8: Save merged dataset
# --------------------
df_merged.to_csv(output_file, index=False)
print(f"✅ Cleaned & merged dataset saved at: {output_file}")
print("Preview:")
print(df_merged.head(20))


In [ ]:
import os
import pandas as pd
from google.colab import drive

# ----------------------
# STEP 1: Mount Drive
# ----------------------
drive.mount('/content/drive')

# ----------------------
# STEP 2: Base folder
# ----------------------
base_dir = "/content/drive/MyDrive/DELHI/GroundTruth_Data"  # 🔁 change if needed

# ----------------------
# STEP 3: Merge station data
# ----------------------
all_stations = []

for station_folder in os.listdir(base_dir):
    station_path = os.path.join(base_dir, station_folder)

    if os.path.isdir(station_path):  # Only enter station directories
        print(f"📂 Processing station: {station_folder}")
        monthly_data = []

        for file in os.listdir(station_path):
            if file.endswith(".csv") or file.endswith(".xlsx"):
                file_path = os.path.join(station_path, file)

                try:
                    # Read CSV/Excel
                    if file.endswith(".csv"):
                        df = pd.read_csv(file_path, skiprows=16)  # skip headers if needed
                    else:
                        df = pd.read_excel(file_path, skiprows=16)

                    # Ensure correct columns exist
                    if "From Date" in df.columns and "NO2" in df.columns:
                        df["date"] = pd.to_datetime(df["From Date"], errors="coerce")
                        df["NO2"] = pd.to_numeric(df["NO2"], errors="coerce")
                        df["Station Name"] = station_folder
                        monthly_data.append(df[["Station Name", "date", "NO2"]])
                    else:
                        print(f"⚠️ Skipping {file} - missing columns")

                except Exception as e:
                    print(f"❌ Error reading {file}: {e}")

        # Merge all 12 months for this station
        if monthly_data:
            station_df = pd.concat(monthly_data, ignore_index=True)
            all_stations.append(station_df)

# ----------------------
# STEP 4: Merge all 39 stations
# ----------------------
cpcb_all = pd.concat(all_stations, ignore_index=True)

# ----------------------
# STEP 5: Clean & save
# ----------------------
cpcb_all.dropna(subset=["date", "NO2"], inplace=True)  # remove invalid rows
cpcb_all.sort_values(["Station Name", "date"], inplace=True)

output_file = "/content/drive/MyDrive/DELHI/cpcb_groundtruth_2024.csv"
cpcb_all.to_csv(output_file, index=False)

print(f"✅ Final CPCB dataset saved at: {output_file}")
print(cpcb_all.head(20))


In [ ]:
import os#IMPORTANT SNIPPET FOR MERGING DATASET
import pandas as pd
from google.colab import drive

# ----------------------
# STEP 1: Mount Drive
# ----------------------
drive.mount('/content/drive')

# ---- CONFIG ----
station_folder = "/content/drive/MyDrive/DELHI/GroundTruth_Data/Rohini"

# Columns in your Excel/CSV files
from_col = "From Date"
to_col   = "To Date"
no2_col  = "NO2"

# ---- STEP 1: Collect all monthly files ----
all_files = [os.path.join(station_folder, f)
             for f in os.listdir(station_folder)
             if f.endswith(".csv") or f.endswith(".xlsx")]

dfs = []
for f in all_files:
    try:
        if f.endswith(".csv"):
            # First 16 rows are junk → keep row 17 as header
            df = pd.read_csv(f, skiprows=15)
        else:
            df = pd.read_excel(f, skiprows=15)

        # Ensure dataframe has at least 3 columns before renaming
        if df.shape[1] >= 3:
            # Ensure column renaming is correct
            df.rename(columns={df.columns[0]: "From Date",
                               df.columns[1]: "To Date",
                               df.columns[2]: "NO2"}, inplace=True)
            dfs.append(df)
        else:
            print(f"⚠️ Skipping file {os.path.basename(f)}: Does not have enough columns.")

    except Exception as e:
        print(f"❌ Error reading file {os.path.basename(f)}: {e}")


# Merge all months together
if dfs: # Check if dfs is not empty
    df_station = pd.concat(dfs, ignore_index=True)

    # ---- STEP 2: Parse datetime ----
    df_station[from_col] = pd.to_datetime(df_station[from_col], errors="coerce")

    # Drop rows where timestamp is missing
    df_station = df_station.dropna(subset=[from_col])

    # ---- STEP 3: Daily average ----
    df_daily = df_station.groupby(df_station[from_col].dt.date)[no2_col].mean().reset_index()

    # Rename columns
    df_daily.rename(columns={from_col: "date", no2_col: "NO2_daily"}, inplace=True)

    # ---- STEP 4: Save ----
    out_path = os.path.join(station_folder, "station_daily_merged.csv")
    df_daily.to_csv(out_path, index=False)

    print(f"✅ Done! Daily averaged file saved at: {out_path}")
else:
    print(f"❌ No valid files found in {station_folder} to process.")

In [ ]:
import os#IMPORTANT SNIPPET FOR MERGING DATASET
import pandas as pd

# Paths
station_folder = "/content/drive/MyDrive/DELHI/GroundTruth_Data/Rohini" # Corrected typo in folder name
station_meta_file = "/content/drive/MyDrive/DELHI/Delhi_weatherst_lat_lon (1).csv"  # <-- your station lat/lon file

# Load the daily dataset created earlier
df_daily = pd.read_csv(os.path.join(station_folder, "station_daily_merged.csv"))

# Load station metadata
df_meta = pd.read_csv(station_meta_file)

# Clean station name for consistency
df_meta["Station Name"] = df_meta["Station Name"].str.strip().str.upper()
station_name = os.path.basename(station_folder).strip().upper()

# Select the correct station metadata row
meta_row = df_meta[df_meta["Station Name"] == station_name].iloc[0]

# Add station info to the dataframe
df_daily.insert(0, "Station Name", meta_row["Station Name"])
df_daily.insert(1, "Latitude", meta_row["Latitude"])
df_daily.insert(2, "Longitude", meta_row["Longitude"])

# Rename NO2 column
df_daily.rename(columns={"NO2_daily": "NO2"}, inplace=True)

# Save final enriched dataset
out_path = os.path.join(station_folder, "station_daily_enriched.csv")
df_daily.to_csv(out_path, index=False)

print(f"✅ Final enriched dataset saved at: {out_path}")

In [ ]:
from google.colab import drive#NOT IN ORDER MERGEING
import pandas as pd
import glob
import os

# Mount Google Drive
drive.mount('/content/drive')

# === CONFIGURATION ===
# Folder where all 12 CPCB Excel files are stored
folder_path = '/content/drive/MyDrive/DELHI/GroundTruth_Data/okhla Phase-2'   # 🔁 update this to your folder path

# Path to your station details CSV (contains Station Name, Latitude, Longitude)
station_csv = '/content/drive/MyDrive/DELHI/Delhi_weatherst_lat_lon (1).csv'  # 🔁 update path

# === LOAD STATION DETAILS ===
station_df = pd.read_csv(station_csv)
station_df['Station Name'] = station_df['Station Name'].str.strip().str.upper()

# === PROCESS ALL EXCEL FILES ===
excel_files = glob.glob(os.path.join(folder_path, '*.xlsx'))
merged_data = []

for file in excel_files:
    filename = os.path.basename(file)
    station_name_guess = filename.split('.')[0].split('_')[0].upper()  # e.g., 'okhlaphase2' → 'OKHLAPHASE2'

    print(f"📄 Processing: {filename}")

    # Read Excel, skip header rows until data starts (around row 17)
    df = pd.read_excel(file, skiprows=16)

    # Keep only relevant columns if they exist
    if 'From Date' not in df.columns or 'NO2' not in df.columns:
        print(f"⚠️ Skipping {filename} (required columns missing)")
        continue

    df = df[['From Date', 'NO2']].dropna()
    # Corrected date format to '%d-%m-%Y %H:%M'
    df['date'] = pd.to_datetime(df['From Date'], format='%d-%m-%Y %H:%M').dt.date

    # Match station name from details file
    matched = station_df[station_df['Station Name'].str.contains(station_name_guess, case=False, na=False)]

    if not matched.empty:
        lat = matched.iloc[0]['Latitude']
        lon = matched.iloc[0]['Longitude']
        station_name = matched.iloc[0]['Station Name']
    else:
        lat, lon, station_name = None, None, station_name_guess

    df['Station Name'] = station_name
    df['Latitude'] = lat
    df['Longitude'] = lon
    df.rename(columns={'NO2': 'no2'}, inplace=True)

    merged_data.append(df[['Station Name', 'Latitude', 'Longitude', 'date', 'no2']])

# === MERGE ALL FILES ===
# === MERGE ALL FILES ===
if merged_data:
    final_df = pd.concat(merged_data, ignore_index=True)
    output_dir = '/content/drive/MyDrive/CPCB_excels'
    os.makedirs(output_dir, exist_ok=True)   # ✅ Create folder if not exists
    output_path = os.path.join(output_dir, 'Delhi_NO2_groundtruth_2024.csv')

    final_df.to_csv(output_path, index=False)
    print(f"✅ Combined dataset saved to:\n{output_path}")
else:
    print("❌ No valid Excel files processed.")


In [ ]:
from google.colab import drive#OKHLA PHASE DATASET MERGE
import pandas as pd
import glob
import os
import re

# === MOUNT DRIVE ===
drive.mount('/content/drive')

# === CONFIGURATION ===
folder_path = '/content/drive/MyDrive/DELHI/GroundTruth_Data/okhla Phase-2'  # folder with all monthly Excel files
station_csv = '/content/drive/MyDrive/DELHI/Delhi_weatherst_lat_lon (1).csv'  # station details CSV
output_dir = '/content/drive/MyDrive/CPCB_excels'
os.makedirs(output_dir, exist_ok=True)

# === LOAD STATION DETAILS ===
station_df = pd.read_csv(station_csv)
station_df['Station Name'] = station_df['Station Name'].str.strip().str.upper()

# === MONTH ORDER MAP ===
month_order = {
    'JAN': 1, 'FEB': 2, 'MAR': 3, 'APR': 4,
    'MAY': 5, 'JUN': 6, 'JUL': 7, 'AUG': 8,
    'SEP': 9, 'OCT': 10, 'NOV': 11, 'DEC': 12
}

# === PROCESS EXCEL FILES ===
excel_files = glob.glob(os.path.join(folder_path, '*.xlsx'))
merged_data = []

for file in excel_files:
    filename = os.path.basename(file)
    print(f"📄 Processing: {filename}")

    # Extract month name from filename (e.g., 'jan_okhlaphase2.xlsx' → 'JAN')
    month_match = re.search(r'(jan|feb|mar|apr|may|jun|jul|aug|sep|oct|nov|dec)', filename, re.IGNORECASE)
    month_name = month_match.group(1).upper() if month_match else 'UNKNOWN'

    # Read Excel (skip headers before data starts)
    df = pd.read_excel(file, skiprows=16)

    if 'From Date' not in df.columns or 'NO2' not in df.columns:
        print(f"⚠️ Skipping {filename} — missing columns")
        continue

    df = df[['From Date', 'NO2']].dropna()
    df['date'] = pd.to_datetime(df['From Date'], format='%d-%m-%Y %H:%M', errors='coerce').dt.date
    df.dropna(subset=['date'], inplace=True)

    # Match station name from details
    matched = station_df[station_df['Station Name'].str.contains('OKHLA', case=False, na=False)]
    if not matched.empty:
        lat, lon, station_name = matched.iloc[0][['Latitude', 'Longitude', 'Station Name']]
    else:
        lat, lon, station_name = None, None, 'OKHLA PHASE-2'

    df['Station Name'] = station_name
    df['Latitude'] = lat
    df['Longitude'] = lon
    df.rename(columns={'NO2': 'no2'}, inplace=True)
    df['Month'] = month_name
    df['Month_Num'] = month_order.get(month_name[:3].upper(), 99)

    merged_data.append(df[['Station Name', 'Latitude', 'Longitude', 'date', 'no2', 'Month', 'Month_Num']])

# === MERGE AND SORT ===
if merged_data:
    final_df = pd.concat(merged_data, ignore_index=True)
    final_df.sort_values(by=['Month_Num', 'date'], inplace=True)
    final_df.drop(columns='Month_Num', inplace=True)

    output_path = os.path.join(output_dir, 'Delhi_NO2_groundtruth_2024.csv')
    final_df.to_csv(output_path, index=False)

    print(f"✅ Combined dataset saved to:\n{output_path}")
    print(f"✅ Rows: {len(final_df)} | Columns: {list(final_df.columns)}")
else:
    print("❌ No valid Excel files processed.")


In [ ]:
import pandas as pd #reordering(DIDN'T WORKED)
from google.colab import drive
import pandas as pd
import glob
import os
import re

# === MOUNT DRIVE ===
drive.mount('/content/drive')

# Load your merged dataset
file_path = '/content/drive/MyDrive/CPCB_excels/Delhi_NO2_groundtruth_2024.csv'  # 🔁 update if needed
df = pd.read_csv(file_path)

# --- Define correct month order ---
month_order = ['JAN', 'FEB', 'MAR', 'APR', 'MAY', 'JUN',
                'JUL', 'AUG', 'SEP', 'OCT', 'NOV', 'DEC']

# --- Fix typo: "FAB" → "FEB" if any ---
df['Month'] = df['Month'].replace({'FAB': 'FEB', 'FEBRUARY': 'FEB'})

# --- Create sort key and reorder ---
df['Month_Num'] = df['Month'].map({m: i for i, m in enumerate(month_order, 1)})
df.sort_values(by=['Month_Num', 'date'], inplace=True)
df.drop(columns='Month_Num', inplace=True)

# --- Save reordered version ---
output_path = '/content/drive/MyDrive/CPCB_excels/Delhi_NO2_groundtruth_2024_sorted.csv'
df.to_csv(output_path, index=False)

print(f"✅ Reordered file saved to:\n{output_path}")

In [ ]:
from google.colab import drive#REORDERING OF OKHLA PHASE
import pandas as pd
import os

# === MOUNT DRIVE ===
drive.mount('/content/drive')

# Load your merged dataset
file_path = '/content/drive/MyDrive/CPCB_excels/Delhi_NO2_groundtruth_2024.csv'  # 🔁 update if needed
df = pd.read_csv(file_path)

# --- Ensure 'date' is a proper datetime ---
df['date'] = pd.to_datetime(df['date'], errors='coerce')

# --- Sort chronologically based on date ---
df = df.sort_values(by='date')

# --- Optional: Extract month name (for readability) ---
df['Month'] = df['date'].dt.strftime('%b').str.upper()  # e.g., JAN, FEB, MAR

# --- Save reordered version ---
output_path = '/content/drive/MyDrive/CPCB_excels/Delhi_NO2_groundtruth_2024_sorted.csv'
os.makedirs(os.path.dirname(output_path), exist_ok=True)
df.to_csv(output_path, index=False)

print(f"✅ Reordered chronologically and saved to:\n{output_path}")


In [ ]:
# =====================================
# 1. Mount Google Drive
# =====================================
from google.colab import drive
drive.mount('/content/drive')

# =====================================
# 2. Import libraries
# =====================================
import pandas as pd

# =====================================
# 3. Load datasets
# =====================================
# 🔁 Update file paths as per your Drive folder
groundtruth_path = '/content/drive/MyDrive/DELHI/groundtruth_no2_okhla.csv'   # Ground truth dataset
satellite_path = '/content/drive/MyDrive/no2_data/TROPOMI_NO2_Delhi_Jan2024_clean.csv'  # GEE satellite dataset

gt_df = pd.read_csv(groundtruth_path)
sat_df = pd.read_csv(satellite_path)

# =====================================
# 4. Standardize and clean columns
# =====================================
# Rename columns for consistency
gt_df.rename(columns={
    'Station Name': 'station',
    'Latitude': 'lat',
    'Longitude': 'lon',
    'date': 'date',
    'no2': 'groundtruth_no2'
}, inplace=True)

sat_df.rename(columns={
    'name': 'station',
    'tropospheric_NO2_column_number_density': 'tropospheric_no2'
}, inplace=True)

# Convert date columns to datetime
gt_df['date'] = pd.to_datetime(gt_df['date'])
sat_df['date'] = pd.to_datetime(sat_df['date'])

# =====================================
# 5. Merge datasets based on date
# =====================================
merged = pd.merge(gt_df, sat_df[['date', 'tropospheric_no2']], on='date', how='left')

# =====================================
# 6. Reorder columns for clarity
# =====================================
final_df = merged[['station', 'lat', 'lon', 'date', 'groundtruth_no2', 'tropospheric_no2']]

# =====================================
# 7. Save to Google Drive
# =====================================
output_path = '/content/drive/MyDrive/no2_data/merged_okhla_no2_2024.csv'
final_df.to_csv(output_path, index=False)

print(f"✅ Merged CSV saved to:\n{output_path}")

# Display first few rows
final_df.head()
